## Docker for pynusmv

In [ ]:
import os

# Pull the image (Docker Desktop)
!docker pull louvainverificationlab/pynusmv

In [ ]:
# Run container
print("Starting Docker container...")
# Specify platform to suppress warning on apple silicon
!docker run --rm --platform linux/amd64 louvainverificationlab/pynusmv python3 -c "import pynusmv; print('PyNuSMV ')"

## Invariant (safety) verification

### Reference Implementation

In [ ]:
%%writefile src/ref_inv_mc.py
import pynusmv
import sys

def spec_to_bdd(model, spec):
    """
    Return the set of states of `model` satisfying `spec`, as a BDD.
    """
    bddspec = pynusmv.mc.eval_ctl_spec(model, spec)
    return bddspec

def check_explain_inv_spec(spec):
    """
    Return whether the loaded SMV model satisfies or not the invariant
    `spec`, that is, whether all reachable states of the model satisfies `spec`
    or not. Return also an explanation for why the model does not satisfy
    `spec``, if it is the case, or `None` otherwise.

    The result is a tuple where the first element is a boolean telling
    whether `spec` is satisfied, and the second element is either `None` if the
    first element is `True``, or an execution of the SMV model violating `spec`
    otherwise.

    The execution is a tuple of alternating states and inputs, starting
    and ennding with a state. States and inputs are represented by dictionaries
    where keys are state and inputs variable of the loaded SMV model, and values
    are their value.
    """
    ltlspec = pynusmv.prop.g(spec)
    res, trace = pynusmv.mc.check_explain_ltl_spec(ltlspec)
    return res, trace

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage:", sys.argv[0], "filename.smv")
        sys.exit(1)

    pynusmv.init.init_nusmv()
    filename = sys.argv[1]
    pynusmv.glob.load_from_file(filename)
    pynusmv.glob.compute_model()
    invtype = pynusmv.prop.propTypes['Invariant']
    for prop in pynusmv.glob.prop_database():
        spec = prop.expr
        if prop.type == invtype:
            print("Property", spec, "is an INVARSPEC.")
            res, trace = check_explain_inv_spec(spec)
            if res == True:
                print("Invariant is respected")
            else:
                print("Invariant is not respected")
                print(trace)
        else:
            print("Property", spec, "is not an INVARSPEC, skipped.")

    pynusmv.init.deinit_nusmv()


### Custom Implementation

#### Outline

The method `check_explain_inv_spec` implements a form of symbolic search, that uses symbolic Breadth-First Search to explore the state space induced by the model, and visit all reachable state regions. The goal is to prove that there are no reachable states (from the initial ones) that violate the specification, by reaching a fix-point without encountering any violation, or to provide a counter-example (i.e. an execution trace that starts in an initial state and terminates in a _bad_ state) build by back-tracking after reaching a region containing a _bad_ state.

#### Details

The method starts by obtaining the BDD corresponding to all the possible states violating the specification, by negation of the BDD representing the compliant states.

Then, we initialize the core variables that keep track of all the reached states (`reached`), the states that are currently being explored (`current_states`), and a list of "state layers" in order of exploration that we need to obtain a witness in case we detect a violation (`trace_layers`).

At this point we start iterating until we either cannot reach new states (fix-point) and thus we have explored all the reachable stated, or we reach a region where there are states violating the spec.
To proceed in the search, in the body of the loop we need to compute the region containing all the successors of the current one using the $Post$ operator.
We then update `current_states` with the successors minus the states already explored, and update the `reached` variable with the new ones. Also, we append to the `trace_layers` list the new region.

After exiting the loop we can check if we reached a fix-point (i.e. visited all reachable states), if so we can return `True`, because no reachable state violates the specification.

Otherwise, we need to build a witness trace.
First, we pick an arbitrary state from the final region that violates the spec and use it to initialize the trace (which is represented as a list).
The trace has shape $[s_0, i_0, s_1, i_1, \dots, s_n]$, where $s_0 \in Init$.

Then, we iterate backwards from the regions stored in `trace_layers` (whose length corresponds to the length of the trace), and use the $Pre$ operator (dual of $Post$), which returns the region of all the possible predecessors of a set of states.
The region returned by $Pre$ is intersected with the current layer, this lets us pick only the states at the proper depth in the traversal tree, avoiding loops, and picking the shortest trace from the root to the _bad_ state.
Finally, we pick a random suitable predecessor, add it to the head of the trace (eventually with the inputs that were part of the transition), and continue with the iterations.

After the loop we return the tuple (`False`, `counterexample`).

#### Correctness

The correctness of the implementation relies on the properties of Symbolic Model Checking and on the sound and exhaustive nature of the search algorithm employed.

The algorithm computes the set of all reachable states ($R$) starting from the initial states ($I$) using Least Fixed-Point iteration. Since the state space of the Finite State Machine is finite and the set of reached states grows monotonically with every iteration, the loop is guaranteed to terminate (completeness).
Upon termination, the set $R$ contains exactly every state that can be reached by the system. By intersecting $R$ with the set of states violating the specification ($\neg Spec$), the algorithm guarantees that if the intersection is empty, the system is provably safe. Conversely, if the intersection is non-empty, a violation exists (soundness).

The construction of the counter-example ensures that the reported trace is both a valid execution of the system and a shortest path to the violation.
By storing the "frontier" of newly discovered states at each step of the forward traversal (`trace_layers`), the algorithm effectively builds a Breadth-First Search (BFS) tree. During the backward reconstruction, the algorithm selects predecessors strictly from the immediate previous layer. By using the intersection with the layers obtained during forward discovery, we avoid cycles. 

For systems with external inputs, the algorithm correctly identifies the specific input vector required to trigger each transition in the trace. This ensures that the counter-example is fully reproducible and full captures the entire transition $(Current State, Inputs, Next State)$.

##### Empirical Testing
To further validate the correctness, the custom implementation was tested against the reference provided. The execution shows complete agreement between the two implementations across the provided test suite, for both the boolean result and the trace.
Actually, our implementation provides a more concise trace that terminates exactly in the violating state, instead the LTL checker of NuSMV returns a longer trace with a loop at the end (which is the more general approach for LTL properties).

In [ ]:
%%writefile src/inv_mc.py
# slightly different approach
import pynusmv
import sys
from typing import Tuple, Optional, Dict, List, Any

def spec_to_bdd(model, spec):
    # Return the set of states of `model` satisfying `spec`, as a BDD.
    return pynusmv.mc.eval_ctl_spec(model, spec)

def check_explain_inv_spec(spec) -> Tuple[bool, Optional[Tuple[Dict[str, str], ...]]]:
    # Check whether the loaded SMV model satisfies the invariant `spec`.
    # Returns: (result, trace)
    fsm_model = pynusmv.glob.prop_database().master.bddFsm

    spec_as_bdd = spec_to_bdd(fsm_model, spec)
    negated_spec = ~spec_as_bdd

    def satisfy_spec(states):
        # Check if all states in the given BDD satisfy the invariant.
        return states.intersection(negated_spec).is_false()

    reached = fsm_model.init
    current_states = fsm_model.init
    trace_layers = [reached]

    # Keep exploring state space until either there are no new successors or
    # there are states violating the spec
    while not current_states.is_false() and satisfy_spec(current_states):
        successors = fsm_model.post(current_states)
        current_states = successors - reached
        reached = reached + current_states
        # Add the visited successive regions to a list so we can do backtracking later
        trace_layers.append(current_states)

    # Exited the loop because we reached a fixpoint => The spec is an invariant
    if satisfy_spec(current_states):
        return True, None

    # There are states violating the property. We need to find a witness.
    invalid_states = current_states.intersection(negated_spec)
    last_state = fsm_model.pick_one_state(invalid_states)

    # Initialize witness trace
    counterexample = [last_state.get_str_values()]
    has_inputs = len(fsm_model.bddEnc.inputsVars) > 0
    next_state = last_state

    # Backtrack using the traversed layers during forward traversal
    for layer in reversed(trace_layers[:-1]):
        # Intersect chosen state predecessors with the layers at depth n-1, so we avoid entering loops and
        # we get a shortest trace from init to a final state violating the spec.
        possible_predecessors = fsm_model.pre(next_state)
        intersect = layer.intersection(possible_predecessors)

        # Can't happen in a valid back-traversal
        if intersect.is_false():
            continue

        chosen_pre = fsm_model.pick_one_state(intersect)

        # Pick any set of inputs (if some) that resulted in a transaction between the chosen predecessor and the current state.
        if has_inputs:
            inputs_between = fsm_model.get_inputs_between_states(chosen_pre, next_state)
            picked_inputs = fsm_model.pick_one_inputs(inputs_between)
            # Cover the case where there were no picked inputs (can't happen for a valid transition)
            if picked_inputs:
                counterexample.insert(0, picked_inputs.get_str_values())
            else:
                counterexample.insert(0, {})
        else:
            counterexample.insert(0, {})

        counterexample.insert(0, chosen_pre.get_str_values())

        next_state = chosen_pre

    # Return the witness execution
    return False, tuple(counterexample)

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage:", sys.argv[0], "filename.smv")
        sys.exit(1)

    pynusmv.init.init_nusmv()
    filename = sys.argv[1]

    try:
        pynusmv.glob.load_from_file(filename)
    except OSError as e:
        print(f"Error: Could not load file '{filename}': {e}")
        pynusmv.init.deinit_nusmv()
        sys.exit(1)

    pynusmv.glob.compute_model()
    invtype = pynusmv.prop.propTypes['Invariant']
    for prop in pynusmv.glob.prop_database():
        spec = prop.expr
        if prop.type == invtype:
            print("Property", spec, "is an INVARSPEC.")
            res, trace = check_explain_inv_spec(spec)
            if res == True:
                print("Invariant is respected")
            else:
                print("Invariant is not respected")
                print(trace)
        else:
            print("Property", spec, "is not an INVARSPEC, skipped.")

    pynusmv.init.deinit_nusmv()

### Execution Code

In [ ]:
import os
import glob
import subprocess

current_dir = os.getcwd()

files_to_check = glob.glob(os.path.join("test", "safety", "*.smv"))

print(f"Starting batch processing of {len(files_to_check)} files...\n")

# Loop through every file
for file_path in files_to_check:
    relative_path = os.path.relpath(file_path, current_dir).replace("\\", "/")

    print(f"Running invariant check on: {os.path.basename(file_path)}")

    # Docker Command
    # -v "{current_dir}:/work"  -> Mounts Windows folder to /work inside Linux
    # -w /work                  -> Makes /work the current directory inside Linux
    # python3 inv_mc.py ...     -> Runs script

    cmd = [
        "docker", "run", "--rm",
        "--platform", "linux/amd64",
        "-v", f"{current_dir}:/work",
        "-w", "/work",
        "louvainverificationlab/pynusmv",
        "python3", "src/inv_mc.py", relative_path
    ]

    ref_cmd = [
        "docker", "run", "--rm",
        "--platform", "linux/amd64",
        "-v", f"{current_dir}:/work",
        "-w", "/work",
        "louvainverificationlab/pynusmv",
        "python3", "src/ref_inv_mc.py", relative_path
    ]

    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        ref_result = subprocess.run(ref_cmd, capture_output=True, text=True)
        print('--- CUSTOM IMPLEMENTATION ---')
        print(result.stdout)
        print('--- REFERENCE IMPLEMENTATION ---')
        print(ref_result.stdout)
        if result.stderr:
            print("Errors:", result.stderr)
    except Exception as e:
        print(f"Failed to run Docker: {e}")

print("\nAll checks finished.")

## Response Properties


### Reference Implementation

In [ ]:
%%writefile src/ref_response_mc.py

import sys
import pynusmv
from pynusmv_lower_interface.nusmv.parser import parser 

specTypes = {'LTLSPEC': parser.TOK_LTLSPEC, 'CONTEXT': parser.CONTEXT,
    'IMPLIES': parser.IMPLIES, 'IFF': parser.IFF, 'OR': parser.OR, 'XOR': parser.XOR, 'XNOR': parser.XNOR,
    'AND': parser.AND, 'NOT': parser.NOT, 'ATOM': parser.ATOM, 'NUMBER': parser.NUMBER, 'DOT': parser.DOT,

    'NEXT': parser.OP_NEXT, 'OP_GLOBAL': parser.OP_GLOBAL, 'OP_FUTURE': parser.OP_FUTURE,
    'UNTIL': parser.UNTIL,
    'EQUAL': parser.EQUAL, 'NOTEQUAL': parser.NOTEQUAL, 'LT': parser.LT, 'GT': parser.GT,
    'LE': parser.LE, 'GE': parser.GE, 'TRUE': parser.TRUEEXP, 'FALSE': parser.FALSEEXP
}

basicTypes = {parser.ATOM, parser.NUMBER, parser.TRUEEXP, parser.FALSEEXP, parser.DOT,
              parser.EQUAL, parser.NOTEQUAL, parser.LT, parser.GT, parser.LE, parser.GE}
booleanOp = {parser.AND, parser.OR, parser.XOR, parser.XNOR, parser.IMPLIES, parser.IFF}

def spec_to_bdd(model, spec):
    """
    Given a formula `spec` with no temporal operators, returns a BDD equivalent to
    the formula, that is, a BDD that contains all the states of `model` that
    satisfy `spec`
    """
    bddspec = pynusmv.mc.eval_simple_expression(model, str(spec))
    return bddspec
    
def is_boolean_formula(spec):
    """
    Given a formula `spec`, checks if the formula is a boolean combination of base
    formulas with no temporal operators. 
    """
    if spec.type in basicTypes:
        return True
    if spec.type == specTypes['NOT']:
        return is_boolean_formula(spec.car)
    if spec.type in booleanOp:
        return is_boolean_formula(spec.car) and is_boolean_formula(spec.cdr)
    return False
    
def parse(spec):
    """
    Visit the syntactic tree of the formula `spec` to check if it is a response formula,
    that is wether the formula is of the form
    
                    G (f -> F g)
    
    where f and g are boolean combination of basic formulas.
    
    If `spec` is a response formula, the result is a pair where the first element is the 
    formula f and the second element is the formula g. If `spec` is not a response 
    formula, then the result is None.
    """
    # the root of a spec should be of type CONTEXT
    if spec.type != specTypes['CONTEXT']:
        return None
    # the right child of a context is the main formula
    spec = spec.cdr
    # the root of a response formula should be of type OP_GLOBAL 
    if spec.type != specTypes['OP_GLOBAL']:
        return None
    # The formula in the scope of the G operator must be of type IMPLIES
    spec = spec.car
    if spec.type != specTypes['IMPLIES']:
        return None
    # Check if lhs of the implication is a formula with no temporal operators
    f_formula = spec.car
    if not is_boolean_formula(f_formula):
        return None
    # Check if rhs of the implication is a F g formula
    g_formula = spec.cdr
    if g_formula.type != specTypes['OP_FUTURE']:
        return None
    g_formula = g_formula.car
    if not is_boolean_formula(g_formula):
        return None
    return (f_formula, g_formula)

def check_explain_response_spec(spec):
    """
    Return whether the loaded SMV model satisfies or not the GR(1) formula
    `spec`, that is, whether all executions of the model satisfies `spec`
    or not. 
    """
    if parse(spec) == None:
        return None
    return pynusmv.mc.check_explain_ltl_spec(spec)

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage:", sys.argv[0], "filename.smv")
        sys.exit(1)
    
    pynusmv.init.init_nusmv()
    filename = sys.argv[1]
    pynusmv.glob.load_from_file(filename)
    pynusmv.glob.compute_model()
    type_ltl = pynusmv.prop.propTypes['LTL']
    for prop in pynusmv.glob.prop_database():
        spec = prop.expr
        print(spec)
        if prop.type != type_ltl:
            print("property is not LTLSPEC, skipping")
            continue
        res = check_explain_response_spec(spec)
        if res == None:
            print('Property is not a response formula, skipping')
        elif res[0] == True:
            print("Property is respected")
        elif res[0] == False:
            print("Property is not respected")
            print("Counterexample:", res[1])
    
    pynusmv.init.deinit_nusmv()


### Custom Implementation (Alt. version)

In [ ]:
%%writefile src/response_mc.py

import sys
import pynusmv
from pynusmv_lower_interface.nusmv.parser import parser 

specTypes = {'LTLSPEC': parser.TOK_LTLSPEC, 'CONTEXT': parser.CONTEXT,
    'IMPLIES': parser.IMPLIES, 'IFF': parser.IFF, 'OR': parser.OR, 'XOR': parser.XOR, 'XNOR': parser.XNOR,
    'AND': parser.AND, 'NOT': parser.NOT, 'ATOM': parser.ATOM, 'NUMBER': parser.NUMBER, 'DOT': parser.DOT,

    'NEXT': parser.OP_NEXT, 'OP_GLOBAL': parser.OP_GLOBAL, 'OP_FUTURE': parser.OP_FUTURE,
    'UNTIL': parser.UNTIL,
    'EQUAL': parser.EQUAL, 'NOTEQUAL': parser.NOTEQUAL, 'LT': parser.LT, 'GT': parser.GT,
    'LE': parser.LE, 'GE': parser.GE, 'TRUE': parser.TRUEEXP, 'FALSE': parser.FALSEEXP
}

basicTypes = {parser.ATOM, parser.NUMBER, parser.TRUEEXP, parser.FALSEEXP, parser.DOT,
              parser.EQUAL, parser.NOTEQUAL, parser.LT, parser.GT, parser.LE, parser.GE}
booleanOp = {parser.AND, parser.OR, parser.XOR, parser.XNOR, parser.IMPLIES, parser.IFF}

def spec_to_bdd(model, spec):
    """
    Given a formula `spec` with no temporal operators, returns a BDD equivalent to
    the formula, that is, a BDD that contains all the states of `model` that
    satisfy `spec`
    """
    bddspec = pynusmv.mc.eval_simple_expression(model, str(spec))
    return bddspec
    
def is_boolean_formula(spec):
    """
    Given a formula `spec`, checks if the formula is a boolean combination of base
    formulas with no temporal operators. 
    """
    if spec.type in basicTypes:
        return True
    if spec.type == specTypes['NOT']:
        return is_boolean_formula(spec.car)
    if spec.type in booleanOp:
        return is_boolean_formula(spec.car) and is_boolean_formula(spec.cdr)
    return False
    
def parse(spec):
    """
    Visit the syntactic tree of the formula `spec` to check if it is a response formula,
    that is wether the formula is of the form
    
                    G (f -> F g)
    
    where f and g are boolean combination of basic formulas.
    
    If `spec` is a response formula, the result is a pair where the first element is the 
    formula f and the second element is the formula g. If `spec` is not a response 
    formula, then the result is None.
    """
    # the root of a spec should be of type CONTEXT
    if spec.type != specTypes['CONTEXT']:
        return None
    # the right child of a context is the main formula
    spec = spec.cdr
    # the root of a response formula should be of type OP_GLOBAL 
    if spec.type != specTypes['OP_GLOBAL']:
        return None
    # The formula in the scope of the G operator must be of type IMPLIES
    spec = spec.car
    if spec.type != specTypes['IMPLIES']:
        return None
    # Check if lhs of the implication is a formula with no temporal operators
    f_formula = spec.car
    if not is_boolean_formula(f_formula):
        return None
    # Check if rhs of the implication is a F g formula
    g_formula = spec.cdr
    if g_formula.type != specTypes['OP_FUTURE']:
        return None
    g_formula = g_formula.car
    if not is_boolean_formula(g_formula):
        return None
    return (f_formula, g_formula)

def check_explain_response_spec(spec):
    """
    Return whether the loaded SMV model satisfies or not the GR(1) formula
    `spec`, that is, whether all executions of the model satisfies `spec`
    or not. 
    """
    # Parse the formula to check whether they are a response formula
    formulas = parse(spec)
    if formulas == None:
        return None
    f, g = formulas

    # We only need to check a response property G(f -> F g), where f and g are simple expressions with
    # no temporal operators. Hence we need to check that the negation F(f & G(!g)) happens.
    # If this is the case, then we know that the system does not satisfy the spec and we need to return a witness.

    # Get the model
    model = pynusmv.glob.prop_database().master.bddFsm

    # Obtain all states of the model that satisfy f and g
    states_f = spec_to_bdd(model, f)
    states_not_g = spec_to_bdd(model, ~g)

    # Now compute all reachable states, starting from init
    reach = model.init
    current = model.init
    reach_frontiers = [model.init]

    while not (model.post(current) - reach).is_false():
        new = model.post(current) - reach
        reach_frontiers.append(new)
        reach = reach + new
        current = new

    # At this point get all reachable states where f is satisfied and g is not.
    # This region will be our starting point for cycle detection
    reach_states_f_not_g = reach & states_f & states_not_g
    if reach_states_f_not_g.is_false(): # Trivially holds
        return True, None

    # We now need to check if, starting from the found states, there are cycles where g never holds
    # To do so we keep expanding the recur set until we reach a fix-point or the empty set
    recur = reach_states_f_not_g
    compliant = True
    while not recur.is_false() and compliant:
        pre_reach = pynusmv.dd.BDD.false() # empty set
        new = model.pre(recur) & states_not_g
        while not new.is_false():
            pre_reach = pre_reach + new
            if recur.entailed(pre_reach): # recur is included in pre_reach
                compliant = False # Recur is a fix-point => There is a cycle where in all states not g holds
                break
            new = (model.pre(new) & states_not_g) - pre_reach
        recur = recur & pre_reach

    if compliant:
        return True, None

    # We have found a violation, we now need to provide a witness.
    has_inputs = len(model.bddEnc.inputsVars) > 0
    
    # First we need to find a state in the obtained recur such that we can build a loop to 
    # itself only using states in PreReach, keeping track of frontiers
    candidates = recur # need to remove failed attempts 
    s = model.pick_one_state(candidates)
    found = False
    while not found:
        frontiers = [] # Reset frontiers for each attempt
        r = pynusmv.dd.BDD.false()
        new = model.post(s) & pre_reach
        while not new.is_false():
            frontiers.append(new)
            r = r + new
            new = model.post(new) & pre_reach
            new = new - r
        r = r & recur
        if s.entailed(r): # s in R
            found = True
        else:
            s = model.pick_one_state(r)

    
    # Now we find the frontier to which the found s belongs, and going backwards
    # we then build a loop to itself
    k = 0
    for i, frontier in enumerate(frontiers):
        if s.entailed(frontier): # s in frontier
            k = i
            break
    # Go back from s to s using the frontiers
    trace = [ s.get_str_values() ]
    current_state = s
    for frontier in reversed(frontiers[:k]): # exclude the k-th element where s already is
        predecessors = model.pre(current_state) & frontier
        pred = model.pick_one_state(predecessors)
        if has_inputs:
            inputs = model.get_inputs_between_states(pred, current_state)
            picked_inputs = model.pick_one_inputs(inputs)
            # Cover the case where there were no picked inputs (can't happen for a valid transition)
            trace.insert(0, picked_inputs.get_str_values() if picked_inputs else {})
        else:
            trace.insert(0, {})
        trace.insert(0, pred.get_str_values())
        current_state = pred
    # finally append s at the end (start) of the loop
    if has_inputs:
        inputs = model.get_inputs_between_states(s, current_state)
        picked_inputs = model.pick_one_inputs(inputs)
        # Cover the case where there were no picked inputs (can't happen for a valid transition)
        trace.insert(0, picked_inputs.get_str_values() if picked_inputs else {})
    else:
        trace.insert(0, {})
    trace.insert(0, s.get_str_values())

    # Now we are only left with building a trace from Init to s
    # First find to which reachability frontier s belongs
    k = 0
    for i, frontier in enumerate(reach_frontiers):
        if s.entailed(frontier): # s in frontier
            k = i
            break
    current_state = s
    for frontier in reversed(reach_frontiers[:k]): # exclude the k-th element where s already is
        predecessors = model.pre(current_state) & frontier
        pred = model.pick_one_state(predecessors)
        if has_inputs:
            inputs = model.get_inputs_between_states(pred, current_state)
            picked_inputs = model.pick_one_inputs(inputs)
            # Cover the case where there were no picked inputs (can't happen for a valid transition)
            trace.insert(0, picked_inputs.get_str_values() if picked_inputs else {})
        else:
            trace.insert(0, {})
        trace.insert(0, pred.get_str_values())
        current_state = pred

    return False, tuple(trace)

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage:", sys.argv[0], "filename.smv")
        sys.exit(1)
    
    pynusmv.init.init_nusmv()
    filename = sys.argv[1]
    pynusmv.glob.load_from_file(filename)
    pynusmv.glob.compute_model()
    type_ltl = pynusmv.prop.propTypes['LTL']
    for prop in pynusmv.glob.prop_database():
        spec = prop.expr
        print(spec)
        if prop.type != type_ltl:
            print("property is not LTLSPEC, skipping")
            continue
        res = check_explain_response_spec(spec)
        if res == None:
            print('Property is not a response formula, skipping')
        elif res[0] == True:
            print("Property is respected")
        elif res[0] == False:
            print("Property is not respected")
            print("Counterexample:", res[1])
    
    pynusmv.init.deinit_nusmv()


### Custom Implementation

In [ ]:
%%writefile src/response_mc.py
# minor modification of Attempt 4
import pynusmv
import sys
from pynusmv_lower_interface.nusmv.parser import parser
from typing import Tuple, Optional, Dict, List, Any

specTypes = {'LTLSPEC': parser.TOK_LTLSPEC, 'CONTEXT': parser.CONTEXT,
             'IMPLIES': parser.IMPLIES, 'IFF': parser.IFF, 'OR': parser.OR, 'XOR': parser.XOR, 'XNOR': parser.XNOR,
             'AND': parser.AND, 'NOT': parser.NOT, 'ATOM': parser.ATOM, 'NUMBER': parser.NUMBER, 'DOT': parser.DOT,
             'NEXT': parser.OP_NEXT, 'OP_GLOBAL': parser.OP_GLOBAL, 'OP_FUTURE': parser.OP_FUTURE,
             'UNTIL': parser.UNTIL,
             'EQUAL': parser.EQUAL, 'NOTEQUAL': parser.NOTEQUAL, 'LT': parser.LT, 'GT': parser.GT,
             'LE': parser.LE, 'GE': parser.GE, 'TRUE': parser.TRUEEXP, 'FALSE': parser.FALSEEXP
             }

basicTypes = {parser.ATOM, parser.NUMBER, parser.TRUEEXP, parser.FALSEEXP, parser.DOT,
              parser.EQUAL, parser.NOTEQUAL, parser.LT, parser.GT, parser.LE, parser.GE}
booleanOp = {parser.AND, parser.OR, parser.XOR,
             parser.XNOR, parser.IMPLIES, parser.IFF}

def spec_to_bdd(model, spec):
    # expression (no temporal operators) -> BDD
    return pynusmv.mc.eval_simple_expression(model, str(spec))

def is_boolean_formula(spec):
    # Checks if spec is a boolean combination of basic formulas
    if spec.type in basicTypes:
        return True
    if spec.type == specTypes['NOT']:
        return is_boolean_formula(spec.car)
    if spec.type in booleanOp:
        return is_boolean_formula(spec.car) and is_boolean_formula(spec.cdr)
    return False

def parse(spec):
    # Parser for Response properties: G(f -> F g)
    # Returns (f_formula, g_formula) if 'spec' matches, else None
    if spec.type != specTypes['CONTEXT']:
        return None
    spec = spec.cdr

    if spec.type != specTypes['OP_GLOBAL']: # must be G(...)
        return None
    spec = spec.car

    if spec.type != specTypes['IMPLIES']: # must be f -> ...
        return None

    f_formula = spec.car
    if not is_boolean_formula(f_formula):
        return None

    g_formula = spec.cdr
    if g_formula.type != specTypes['OP_FUTURE']: # must be F g
        return None
    g_formula = g_formula.car

    if not is_boolean_formula(g_formula):
        return None

    return (f_formula, g_formula)

def _compute_reachable(fsm_model) -> Tuple[pynusmv.dd.BDD, List[pynusmv.dd.BDD]]:
    # Set of reachable states (R) using forward BFS
    # Returns (R, trace_layers)
    R = fsm_model.init
    F = fsm_model.init
    trace_layers = [F]

    while not F.is_false():
        F_next = fsm_model.post(F) - R
        if F_next.is_false():
            break
        trace_layers.append(F_next)
        R = R + F_next
        F = F_next

    return R, trace_layers

def _compute_eg(fsm_model, bdd_prop) -> pynusmv.dd.BDD:
    # Set of states satisfying EG(bdd_prop).
    # Fixed Point: Z = bdd_prop & EX(Z)
    Z = bdd_prop.true()      # universal BDD True
    while True:
        pre_Z = fsm_model.pre(Z)
        next_Z = bdd_prop & pre_Z
        if next_Z == Z:
            return Z
        Z = next_Z


def _build_prefix_trace(fsm_model, trace_layers, violating_state_bdd) -> List[Dict[str, str]]:
    # Prefix of a counterexample from Init to violating_state_bdd
    last_state = violating_state_bdd
    counterexample_list = [last_state.get_str_values()]

    has_inputs = len(fsm_model.bddEnc.inputsVars) > 0
    curr_state_bdd = last_state
    backward_state_bdd_list=[last_state]

    # Iterate backwards to find the path
    for layer in reversed(trace_layers[:-1]):
        # predecessor of 'curr' in current 'layer'
        possible_predecessors = fsm_model.pre(curr_state_bdd) & layer

        if possible_predecessors.is_false():
            continue

        prev_state_bdd = fsm_model.pick_one_state(possible_predecessors)

        if has_inputs:
            inputs_between = fsm_model.get_inputs_between_states(prev_state_bdd, curr_state_bdd)
            picked_inputs = fsm_model.pick_one_inputs(inputs_between)
            counterexample_list.insert(0, picked_inputs.get_str_values())
        else:
            counterexample_list.insert(0, {})

        counterexample_list.insert(0, prev_state_bdd.get_str_values())
        backward_state_bdd_list.append(prev_state_bdd)
        curr_state_bdd = prev_state_bdd

    return counterexample_list, backward_state_bdd_list

def check_explain_response_spec(spec):
    # Checks a Response property G(f -> F g)
    fsm_model = pynusmv.glob.prop_database().master.bddFsm

    parsed = parse(spec)
    if parsed is None:
        return None

    f_formula, g_formula = parsed

    bdd_f = spec_to_bdd(fsm_model, f_formula) # may include inputs
    bdd_g = spec_to_bdd(fsm_model, g_formula)
    bdd_not_g = ~bdd_g

    bdd_eg_not_g = _compute_eg(fsm_model, bdd_not_g)

    # abstract input vars from bdd_f to fix switch 1/2
    inputs_vars = getattr(fsm_model.bddEnc, 'inputsVars', None)
    bdd_f_states = bdd_f # default if no inputs

    if inputs_vars and len(inputs_vars) > 0:
        inputs_cube = None
        try:
            inputs_cube = fsm_model.bddEnc.cube_for_inputs_vars()
        except Exception:
            inputs_cube = getattr(fsm_model.bddEnc, 'inputsCube', None)
            if callable(inputs_cube):
                try:
                    inputs_cube = inputs_cube()
                except Exception:
                    pass

        tried = False
        last_exc = None
        if inputs_cube is not None:
            try:
                bdd_f_states = bdd_f.forsome(inputs_cube)
                tried = True
            except Exception as e:
                last_exc = e
                tried = False
        if not tried:
            try:
                bdd_f_states = bdd_f.forsome(inputs_vars)
                tried = True
            except Exception as e:
                last_exc = e
                tried = False
        if not tried:
            try:
                bdd_f_states = bdd_f.existAbstract(inputs_cube)
                tried = True
            except Exception:
                try:
                    bdd_f_states = bdd_f.exist(inputs_cube)
                    tried = True
                except Exception as e:
                    last_exc = e
                    tried = False

        if not tried:
            raise RuntimeError("Could not abstract input variables from bdd_f. Last exc: {}".format(repr(last_exc)))

    bdd_bad_states = bdd_f_states & bdd_eg_not_g

    if bdd_bad_states.is_false():
        return (True, None)

    reachable_states, trace_layers = _compute_reachable(fsm_model)

    # Intersect: reachable; pre(EG(¬g)); exists-input-f; ¬g
    pre_into_eg = fsm_model.pre(bdd_eg_not_g)
    candidate_states = reachable_states & pre_into_eg & bdd_f_states & bdd_not_g

    if candidate_states.is_false():
        return (True, None)

    # witness triple (s, input, s_next)
    s_violation_bdd = None
    s_violation_next_bdd = None
    s_violation_input_bdd = None

    remaining = candidate_states
    while not remaining.is_false():
        s = fsm_model.pick_one_state(remaining)
        remaining = remaining & ~s

        # successors of s in EG(¬g)
        possible_successors = fsm_model.post(s) & bdd_eg_not_g
        if possible_successors.is_false():
            continue

        succs = possible_successors
        while not succs.is_false():
            s_next = fsm_model.pick_one_state(succs)
            succs = succs & ~s_next

            # inputs enabling s -> s_next
            try:
                inputs_between = fsm_model.get_inputs_between_states(s, s_next)
            except Exception:
                inputs_between = None

            if inputs_between is None or inputs_between.is_false():
                continue

            # input that allows transition + makes f true at s
            witness_inputs_bdd = bdd_f & s & inputs_between
            if witness_inputs_bdd.is_false():
                continue

            # found witness
            s_violation_bdd = s
            s_violation_next_bdd = s_next
            if hasattr(fsm_model, 'pick_one_inputs'):
                s_violation_input_bdd = fsm_model.pick_one_inputs(witness_inputs_bdd)
            else:
                s_violation_input_bdd = witness_inputs_bdd
            break

        if s_violation_bdd is not None:
            break

    if s_violation_bdd is None:
        return (True, None)

    # prefix trace (Init -> ... -> s_violation)
    prefix_trace_list, backward_state_bdd_list = _build_prefix_trace(fsm_model, trace_layers, s_violation_bdd)

    # loop (s_violation -> ... -> s_loop)
    loop_list = []
    has_inputs = len(fsm_model.bddEnc.inputsVars) > 0

    if has_inputs:
        try:
            picked_inputs_dict = s_violation_input_bdd.get_str_values()
        except Exception:
            # reconstruct inputs between chosen states
            picked_inputs = fsm_model.pick_one_inputs(fsm_model.get_inputs_between_states(s_violation_bdd, s_violation_next_bdd))
            picked_inputs_dict = picked_inputs.get_str_values()
        loop_list.append(picked_inputs_dict)
    else:
        loop_list.append({})

    loop_list.append(s_violation_next_bdd.get_str_values())

    # closing loop
    curr_bdd = s_violation_next_bdd
    bdd_eg_not_g_consec = [backward_state_bdd_list[-1].get_str_values()]
    for state in reversed(backward_state_bdd_list[:-1]):
        if (state & bdd_eg_not_g).is_false():
            break
        bdd_eg_not_g_consec.append(state.get_str_values())

    while True:
        possible_successors = fsm_model.post(curr_bdd) & bdd_eg_not_g
        if possible_successors.is_false():
            # EG promised a successor
            return (False, tuple(prefix_trace_list))

        next_bdd = fsm_model.pick_one_state(possible_successors)
        if has_inputs:
            inputs_between = fsm_model.get_inputs_between_states(curr_bdd, next_bdd)
            picked_inputs = fsm_model.pick_one_inputs(inputs_between)
            loop_list.append(picked_inputs.get_str_values())
        else:
            loop_list.append({})
        loop_list.append(next_bdd.get_str_values())

        if next_bdd.get_str_values() in bdd_eg_not_g_consec:
            full_trace = prefix_trace_list + loop_list
            return (False, tuple(full_trace))

        bdd_eg_not_g_consec.append(next_bdd.get_str_values())
        curr_bdd = next_bdd

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage:", sys.argv[0], "filename.smv")
        sys.exit(1)

    pynusmv.init.init_nusmv()
    filename = sys.argv[1]
    try:
        pynusmv.glob.load_from_file(filename)
        pynusmv.glob.compute_model()
    except OSError as e:
        print(f"Error loading file: {e}")
        pynusmv.init.deinit_nusmv()
        sys.exit(1)

    type_ltl = pynusmv.prop.propTypes['LTL']

    for prop in pynusmv.glob.prop_database():
        spec = prop.expr
        print(spec)

        if prop.type != type_ltl:
            print("property is not LTLSPEC, skipping")
            continue

        res = check_explain_response_spec(spec)

        if res is None:
            print('Property is not a response formula, skipping')
        elif res[0] == True:
            print("Property is respected")
        elif res[0] == False:
            print("Property is not respected")
            print("Counterexample:", res[1])

    pynusmv.init.deinit_nusmv()


In [ ]:
import os
import glob
import subprocess

current_dir = os.getcwd()

files_to_check = glob.glob(os.path.join("test", "response", "*.smv"))

print(f"Starting batch processing of {len(files_to_check)} files...\n")

# Loop through every file
for file_path in files_to_check:
    relative_path = os.path.relpath(file_path, current_dir).replace("\\", "/")

    print(f"Running invariant check on: {os.path.basename(file_path)}")

    # Docker Command
    # -v "{current_dir}:/work"  -> Mounts Windows folder to /work inside Linux
    # -w /work                  -> Makes /work the current directory inside Linux
    # python3 inv_mc.py ...     -> Runs script

    cmd = [
        "docker", "run", "--rm",
        "--platform", "linux/amd64",
        "-v", f"{current_dir}:/work",
        "-w", "/work",
        "louvainverificationlab/pynusmv",
        "python3", "src/response_mc.py", relative_path
    ]

    ref_cmd = [
        "docker", "run", "--rm",
        "--platform", "linux/amd64",
        "-v", f"{current_dir}:/work",
        "-w", "/work",
        "louvainverificationlab/pynusmv",
        "python3", "src/ref_response_mc.py", relative_path
    ]

    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        ref_result = subprocess.run(ref_cmd, capture_output=True, text=True)
        print('--- CUSTOM IMPLEMENTATION ---')
        print(result.stdout)
        print('--- REFERENCE IMPLEMENTATION ---')
        print(ref_result.stdout)
        if result.stderr:
            print("Errors:", result.stderr)
    except Exception as e:
        print(f"Failed to run Docker: {e}")

print("\nAll checks finished.")